In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
root_dir = '/Users/kessapassa/OneDrive/research_log/20190114/'

In [3]:
dir_list = ['people10000', 'people20000', 'people30000']
seed_list = [str(123 + i) for i in range(3)]
csv_list = ['od']
area_list = [str(i) for i in range(81)]
col_names = [ 'c{0:02d}'.format(i) for i in range(30)]

In [4]:
csv_array = {}
for _dir in dir_list:
    csv_array[_dir] = {}
    for _seed in seed_list:
        csv_array[_dir][_seed] = {}
        for _csv in csv_list:
            df = pd.read_csv(root_dir + 'Origin/' + _dir + 'seed' + _seed + '_' + _csv + '.csv',
                                                names=col_names,
                                                encoding='Shift_JISx0213')
            df.replace(' ', np.NaN, inplace=True)
            df.dropna(how='all', axis=1, inplace=True)
            df = df.applymap(lambda x: (x.split('(census)')[0]) if (type(x) is str) and ('(census)' in x) else x)
            csv_array[_dir][_seed][_csv] = df

In [5]:
road_to_area = {}
def create_road_to_area(road, area):
    road_to_area[road] = float(area)
    
for _dir in dir_list:
    for _seed in seed_list:
        for _csv in ['census']:
            df = pd.read_csv(root_dir + 'include_area_-1/' + _dir + 'seed' + _seed + '_' + _csv + '.csv',
                                                encoding='Shift_JISx0213')
            df = df.loc[:, ['road', 'area']]
            for row in np.asanyarray(df):
                create_road_to_area(row[0], row[1])
road_to_area[np.nan] = np.nan

In [6]:
# df = df.dropna(how='all')
# df = df.applymap(lambda x: road_to_area[x])
# df.head()

In [7]:
dellist = lambda items, indexes: [item for index, item in enumerate(items) if index not in indexes]

def remove_and_fill_none_area(series):
    row = series.values.tolist()
    if -1 in row:
        # -1のindexをリストで取得
        index_list = [i for i, x in enumerate(row) if x == -1]
        # popと同じような原理で削除し横詰め
        row = dellist(row, index_list)
        # 削除した分、ズレが生じるので最後尾に消した数のNaNを追加
        row.extend([np.nan for i in range(len(index_list))])
        
        return pd.Series(row, index=series.index)
    else:
        return series

In [8]:
# df['count'] = df.T.apply(lambda x: x.count())
# df = df.sort_values(['count'], ascending=False)
# df.reset_index(drop=True, inplace=True)
# df.head()

In [9]:
for _dir in dir_list:
    for _seed in seed_list:
        for _csv in csv_list:
            df = csv_array[_dir][_seed][_csv].copy()
#             df = df.dropna(how='all')
            df.iloc[:, 3:] = df.iloc[:, 3:].applymap(lambda x: road_to_area[x])
            df = df[df.loc[:, 'c03'] >= 0]
            dfT = df.T
            dfT = dfT.apply(remove_and_fill_none_area)
            df = dfT.T
            df.reset_index(drop=True, inplace=True)
#             df = pd.concat([df_id, df], axis=1)
            df = df.rename(columns={'c00': 'id'})
            df = df.sort_values(['id'])
            df = df.drop(['c01', 'c02'], axis=1)
            csv_array[_dir][_seed][_csv] = df
            df.to_csv(root_dir + 'converted_road_to_area_od/' + _dir + 'seed' + _seed + '_' +  _csv + '.csv', index=False)

In [10]:
erf

NameError: name 'erf' is not defined

In [ ]:
def create_next_move_area(df, area):
    area = float(area)
    df_area = pd.DataFrame(np.zeros((81, 2)), columns=['area', 'people'])
    df_area['area'] = [float(i) for i in range(81)]
    
    for row in df.values.tolist():
        if area in row:
            index = row.index(area)
            if (len(row) > index+1) and (not np.isnan(row[index+1])):
                df_area.loc[df_area['area'] == row[index+1], 'people'] += 1
                    
    return df_area

In [ ]:
move_array = {}
for _dir in dir_list:
    move_array[_dir] = {}
    for _seed in seed_list:
        move_array[_dir][_seed] = {}
        for _csv in csv_list:
            move_array[_dir][_seed][_csv] = {}
            df = csv_array[_dir][_seed][_csv].copy()
            
            for _area in area_list:
                df_area = create_next_move_area(df, _area)
#                 df_area.to_csv(root_dir + 'next_move_area_od/' + _dir + 'seed' + _seed + '_' + _csv + _area + '.csv', index=False)
                move_array[_dir][_seed][_csv][_area] = df_area
            print(_dir + 'seed' + _seed + '_' + _csv)

In [ ]:
def create_2d(df):
    df_base = pd.DataFrame(np.zeros((81, 2)), columns=['area', 'people'])
    df_base['area'] = [i for i in range(81)]

    for row in df.values.tolist():
        row = [x for x in row if not np.isnan(x)]
        for r in row:
            df_base.loc[r, 'people'] += 1
            
    return df_base

In [ ]:
csv_array_2d = {}
for _dir in dir_list:
    csv_array_2d[_dir] = {}
    for _seed in seed_list:
        csv_array_2d[_dir][_seed] = {}
        for _csv in csv_list:
            df = csv_array[_dir][_seed][_csv].copy()
            df = create_2d(df)
            csv_array_2d[_dir][_seed][_csv] = df
#             df.to_csv(root_dir + 'OD_2D/' + _dir + 'seed' + _seed + _csv + '.csv')

In [ ]:
df = pd.read_csv(root_dir + 'OD_2D/' + 'people30000' + 'seed' + '123' + 'od' + '.csv',
                            index_col=0)
df_new = pd.DataFrame()
for i in range(9):
    tmp = df.loc[i*9:i*9+8, 'people']
    tmp.reset_index(drop=True, inplace=True)
    df_new = pd.concat([df_new, tmp], axis=1)
    
df_new = df_new.T
df_new.columns=(['{0:02d}'.format(i*10) for i in range(9)])
df_new.index=([str(i) for i in range(9)])
df_new = df_new.sort_index(ascending=False)
df_new

In [ ]:
for _dir in dir_list:
    for _seed in seed_list:
        for _csv in csv_list:
            df = pd.read_csv(root_dir + 'OD_2D/' + _dir + 'seed' + _seed + _csv + '.csv',
                            index_col=0)
            
            df_new = pd.DataFrame()
            for i in range(9):
                tmp = df.loc[i*9:i*9+8, 'people']
                tmp.reset_index(drop=True, inplace=True)
                df_new = pd.concat([df_new, tmp], axis=1)
            
            df_new = df_new.T
            df_new.columns=(['{0:02d}'.format(i*10) for i in range(9)])
            df_new.index=([str(i) for i in range(9)])
#             df_new = df_new.sort_index(ascending=False)
            df = df_new
            print(_dir, _seed)
            print(df.sum().sum())
#             df.to_csv(root_dir + 'OD_3D/' + _dir + 'seed' + _seed + _csv + '.csv', index=False)

In [ ]:
csv_array_3d = {}
for _dir in dir_list:
    csv_array_3d[_dir] = {}
    for _seed in seed_list:
        csv_array_3d[_dir][_seed] = {}
        for _csv in csv_list:
            df = pd.read_csv(root_dir + 'OD_3D/' + _dir + 'seed' + _seed + _csv + '.csv')
            csv_array_3d[_dir][_seed][_csv] = df

In [ ]:
import seaborn as sns
def create_contour(df, file_name=False):
    plt.figure(figsize=(9,6))
    if file_name:
        plt.figure(figsize=(9,6), dpi=600)

    sns.heatmap(df, annot=True, fmt='.0f', cmap='RdYlGn_r',
               vmax=df.sum().mean(), vmin=0)
    if file_name:
        plt.savefig('{0}.png'.format(file_name))

In [ ]:
people = 'people30000'
seed = '123'
csv = 'od'
df = csv_array_3d[people][seed][csv].copy()
df = df.sort_index(ascending=False)
print(df.sum().sum())
# create_contour(df, '{0}seed{1}_{2}_for_census'.format(people, seed, csv))
create_contour(df)

In [ ]:
csv_array_2d = {}
for _dir in dir_list:
    csv_array_2d[_dir] = {}
    for _seed in seed_list:
        csv_array_2d[_dir][_seed] = {}
        for _csv in csv_list:
            df = pd.read_csv(root_dir + 'OD_2D/' + _dir + 'seed' + _seed + _csv + '.csv', index_col=0)
            csv_array_2d[_dir][_seed][_csv] = df

In [ ]:
def area_to_contour(area_id):
    area_one_length = 9
    area_id = int(area_id)
    left = str(area_id // area_one_length)
    right = str(area_id % area_one_length) + '0'
    return left+right

In [ ]:
df = csv_array_2d[people][seed][csv].copy()
df['area'] = df['area'].apply(area_to_contour)
df.sort_values(['people'], ascending=False)